# Metric to evaluate the cadence of observations over a visibility window

## Motivation:
Science applications often require that repeated observations of a given field in a given filter take place at intervals dictated by the phenomenon being studied.  However in practice these regularity of observations must take the field visibility into account, since intervals will necessarily be longer while the target is unobservable. 

The goal of this metric is to calculate the desired number of observations as a function of time, and to compare this with the achieved number of observations.  

In [3]:
from __future__ import print_function
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles
from lsst.sims.maf.metrics import BaseMetric
import calc_expected_visits

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)
import astropy.units as u
from astropy.time import Time, TimeDelta
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.coordinates import get_sun
from astropy.coordinates import get_moon

In [5]:
obsdb = db.OpsimDatabase('../../tutorials/baseline2018a.db')
outputDir = '/home/docmaf/'
resultsDb = db.ResultsDb(outDir=outputDir)

In [6]:
propids, proptags = obsdb.fetchPropInfo()
ddWhere = obsdb.createSQLWhere('DD',proptags)
print(proptags)
print(propids)

{'WFD': [3], 'DD': [5], 'NES': [1]}
{1: 'NorthEclipticSpur', 2: 'SouthCelestialPole', 3: 'WideFastDeep', 4: 'GalacticPlane', 5: 'DeepDrillingCosmology1'}


In [7]:
for key, value in obsdb.columnNames.items():
    print(key, value)

Config ['configId', 'Session_sessionId', 'paramName', 'paramValue']
Field ['fieldId', 'Session_sessionId', 'fov', 'ra', 'dec', 'gl', 'gb', 'el', 'eb']
ObsExposures ['exposureId', 'Session_sessionId', 'exposureNum', 'exposureStartTime', 'exposureTime', 'ObsHistory_observationId']
ObsHistory ['observationId', 'Session_sessionId', 'night', 'observationStartTime', 'observationStartMJD', 'observationStartLST', 'TargetHistory_targetId', 'Field_fieldId', 'groupId', 'filter', 'ra', 'dec', 'angle', 'altitude', 'azimuth', 'numExposures', 'visitTime', 'visitExposureTime', 'airmass', 'skyBrightness', 'cloud', 'seeingFwhm500', 'seeingFwhmGeom', 'seeingFwhmEff', 'fiveSigmaDepth', 'moonRA', 'moonDec', 'moonAlt', 'moonAz', 'moonDistance', 'moonPhase', 'sunRA', 'sunDec', 'sunAlt', 'sunAz', 'solarElong']
ObsProposalHistory ['propHistId', 'Session_sessionId', 'Proposal_propId', 'proposalValue', 'proposalNeed', 'proposalBonus', 'proposalBoost', 'ObsHistory_observationId']
Proposal ['propId', 'Session_sess

In [8]:
class CadenceOverVisibilityWindowMetric(BaseMetric):
    """Metric to compare the lightcurve cadence produced by LSST over the visibility window 
    for a given position in the sky to the desired cadence"""
    
    def __init__(self, cols=['fieldRA','fieldDec'], metricName='CadenceOverVisibilityWindowMetric',**kwargs):
        """Kwargs must contain:
        filters  list Filterset over which to compute the metric
        cadence  list Cadence desired for each filter in units of decimal hours
        start_date string Start of observing window YYYY-MM-DD
        end_date string End of observing window YYYY-MM-DD
        """
        
        self.ra_col = 'fieldRA'
        self.dec_col = 'fieldDec'
        self.exp_col = 'visitExposureTime'
        self.n_exp_col = 'numExposures'
        self.obstime_col = 'observationStartMJD'
        self.visittime_col = 'visitTime'
        self.metricName = 'CadenceOverVisibilityWindowMetric'
        
        for key in ['filters', 'cadence', 'start_date', 'end_date']:
            if key in kwargs.keys():
                setattr(self, key, kwargs[key])
            else:
                print('ERROR: Missing data for '+key)
                exit()
                
        cols = [ self.ra_col, self.dec_col, self.exp_col, self.n_exp_col, self.obstime_col, self.visittime_col ]
        
        super(CadenceOverVisibilityWindowMetric,self).__init__(col=cols, metricName=metricName,**kwargs)
    
    def run(self, dataSlice, slicePoint=None):
        
        t = np.empty(dataSlice.size, dtype=list(zip(['time','filter'],[float,'|S1'])))
        t['time'] = dataSlice[self.mjdCol]
        
        t_start = Time(start_date+' 00:00:00')
        t_end = Time(end_date+' 00:00:00')
        n_days = int((t_end - t_start).value)
        dates = np.array([t_start + \
                TimeDelta(i,format='jd',scale=None) for i in range(0,n_days,1)])
        
        result = 0.0
        
        for i,f in enumerate(self.filters):
            
            # Array of the number of visits 
            n_visits_desired = calc_expected_visits.calc_expected_visits(dataSlice[self.ra_col],
                                                                         dataSlice[self.dec_col],
                                                                         self.cadence[i],
                                                                         self.start_date,self.end_date)
            
            idx = np.where(dataSlice[self.filterCol] == f)
            actual_visits_per_filter = dataSlice[self.filterCol][idx]
            
            n_visits_actual = 0.0
            for j,d in enumerate(dates):
                
                tdx = np.where(actual_visits_per_filter[self.mjdCol] == (d.jd-2400000.5))
            
                n_visits_actual = float(len(actual_visits_per_filter[tdx]))
            
                night_efficiency = n_visits_actual / n_visits_desired[j]
            
                result += night_efficiency
                
        return result


In [9]:
propids, proptags = obsdb.fetchPropInfo()
wfdWhere = obsdb.createSQLWhere('WFD',proptags)

In [ ]:
params = {'filters': ['g', 'r', 'i', 'z'],
          'cadence': [ 168.0, 168.0, 1.0, 168.0 ],
          'start_date': '2020-03-01',
          'end_date': '2020-04-01'}

metric = CadenceOverVisibilityWindowMetric(**params)
slicer = slicers.HealpixSlicer(nside=64)
sqlconstraint = wfdWhere
bundle = metricBundles.MetricBundle(metric, slicer, sqlconstraint)

In [12]:
bgroup = metricBundles.MetricBundleGroup({0:bundle}, obsdb, outDir='newmetric_test',resultsDb=resultsDb)
bgroup.runAll()

Querying database SummaryAllProps with constraint proposalId = 3 for columns ['fieldRA', 'visitTime', 'numExposures', 'fieldDec', 'observationStartMJD', 'visitExposureTime']
Found 2049326 visits
Running:  [0]


ValueError: shape mismatch: objects cannot be broadcast to a single shape